In [9]:
from tqdm import tqdm
from torch.utils.data.dataset import Dataset
from torch.utils.data.dataloader import DataLoader
import torchvision
import torch
import torch.nn as nn
import pandas as pd
import os
from torchvision import transforms
from PIL import Image
from d2l import torch as d2l

In [13]:
def pre_datadealer(annotations_file):
    df = pd.read_csv(annotations_file)
    label_list=df['label'].tolist()
    labels = set(label_list)
    label2num = {}
    num2label = {}
    i = 0
    for x in labels:
        label2num[x] = i
        num2label[i] = x
        i+=1
    return label2num,num2label
    
#Adjust according to your data location
label2num,num2label=pre_datadealer("H:/classify-leaves/classify-leaves/train.csv")

In [2]:
class Test_data(Dataset):
    def __init__(self,annotations_file,img_dir):
        super().__init__()
        self.test=pd.read_csv(annotations_file)
        self.img_dir = img_dir
        

    def __len__(self):
        return len(self.test)

    def __getitem__(self,idx):
        rela_img_path = self.test.iloc[idx,0]
        img_path = os.path.join(self.img_dir,rela_img_path).replace('\\','/')
        img = transforms.ToTensor()(Image.open(img_path))
        return img

In [14]:
#Adjust according to your data location
model_path='H:/classify-leaves/train_model.pt'
sub_path= "H:/classify-leaves/classify-leaves/sample_submission.csv"
img_dir = "H:/classify-leaves/classify-leaves"
test_csv= "H:/classify-leaves/classify-leaves/test.csv"

net = torchvision.models.resnet34(weights=torchvision.models.ResNet34_Weights.IMAGENET1K_V1)
net.fc = nn.Linear(in_features=512, out_features=176, bias=True)
net.to(torch.device('cuda'))
net.load_state_dict(torch.load(model_path))
sub_dataset=Test_data(test_csv,img_dir)
sub_iter=DataLoader(sub_dataset,batch_size=64,shuffle=False)
res = pd.read_csv(sub_path)
#res.insert(res.shape[1], 'label', 0)
count = 0
print('test start...')
for X in tqdm(sub_iter):
    X=X.to(d2l.try_gpu())
    preds = net(X).detach().argmax(-1).to(torch.device('cpu')).numpy()
    for num in preds:
        res.iloc[count,1]=num2label[num]
        count+=1
res.to_csv(sub_path, index=False)

test start...


100%|████████████████████████████████████████████████████████████████████████████████| 138/138 [02:17<00:00,  1.00it/s]
